In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob, os
import tensorflow as tf
import logging
tf.get_logger().setLevel(logging.ERROR)
import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow.keras.preprocessing import image, image_dataset_from_directory

from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from PIL import Image
from keras.losses import *
from keras.metrics import *
from tensorflow.keras.applications import EfficientNetB7, inception_v3

In [ ]:
benign = '../input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/benign/'
malign = '../input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/malignant/'
normal = '../input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/normal/'

In [ ]:
def load_image(image_path):
    img = tf.keras.utils.load_img(image_path, target_size= (224, 224 ))
    img = image.img_to_array(img)
    img = tf.keras.applications.efficientnet.preprocess_input(img)
    return img

def remove_mask(list_):
    final_list = []
    for i in list_:
        if('mask' in i):
            continue
        else:
            final_list.append(i)
    return final_list

In [ ]:
benign_image_path = os.listdir(benign)
benign_image_path = [benign+x for x in benign_image_path ]
benign_image_path = remove_mask(benign_image_path)


malign_image_path =  os.listdir(malign)
malign_image_path = [malign+x for x in malign_image_path ]
malign_image_path = remove_mask(malign_image_path)


normal_image_path = os.listdir(normal)
normal_image_path = [normal+x for x in normal_image_path ]
normal_image_path = remove_mask(normal_image_path)


In [ ]:
img = np.array(Image.open(benign_image_path[0]))
plt.imshow(img)

In [ ]:
benign_labels = len(benign_image_path) * [0]
malign_labels = len(malign_image_path) * [1]
normal_labels = len(normal_image_path) * [2]

In [ ]:
#combining the dataset
total_paths = benign_image_path  + malign_image_path + normal_image_path
total_labels = benign_labels + malign_labels + normal_labels

In [ ]:
data_images = []
for paths in total_paths:
    data_images.append(load_image(paths))

In [ ]:
data_images= np.array(data_images)
total_labels = np.array(total_labels)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data_images, total_labels, test_size=.10)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
# BUFFER_SIZE = 1000
# dataset = tf.data.Dataset.from_tensor_slices((total_paths,total_labels)).map(load_image)
# dataset = dataset.shuffle(BUFFER_SIZE).batch(1)
# valid_dataset = dataset.take(200) 
# train_dataset = dataset.skip(200)

In [ ]:
def Efficient_Net():
    base_model = tf.keras.applications.EfficientNetB7(include_top=False,weights='imagenet', input_shape= (224, 224,3))
    base_model.trainable = False
    
    x = base_model.output

    # 1st conv block
    x = Conv2D(256, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = GlobalAveragePooling2D(keepdims = True)(x)

    # 2nd conv block
    x = Conv2D(128, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = GlobalAveragePooling2D(keepdims = True)(x)
    
    x = Dense(64, activation = 'relu')(x)
    x = Activation('relu')(x)
    x = Dropout(.2)(x)

    x = Dense(32, activation = 'relu')(x)
    x = Activation('relu')(x)
    x = Dropout(.2)(x)

    x = Dense(3, 'softmax')(x)
    
    final_model = keras.models.Model(inputs = base_model.input, outputs = x)
    
    return final_model

In [ ]:
earlyStop = keras.callbacks.EarlyStopping(patience=60) 
best_model = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', save_best_only=True) 
tqdm_callback = tfa.callbacks.TQDMProgressBar()

with tf.device('/gpu:0'):
    model = Efficient_Net()
    model.compile(optimizer= keras.optimizers.Adam(), loss = keras.losses.sparse_categorical_crossentropy, 
                  metrics = [keras.metrics.SparseCategoricalAccuracy()])

    history = model.fit(x_train, y_train , epochs = 20,validation_data=(x_test, y_test), batch_size=64,verbose=2, callbacks = [earlyStop,best_model, tqdm_callback ])

In [ ]:
hist = history.history

plt.plot(hist['loss'], label=  'loss')
plt.plot(hist['val_loss'], label = 'val_loss')
plt.plot(hist['sparse_categorical_accuracy'], label='accuracy')
plt.plot(hist['val_sparse_categorical_accuracy'], label='val_accuracy')
plt.legend()